
# Parsl Fest 2025: Parallel Notebook Testing in QMCPy with Parsl

Joshua Herman and Sou-Cheng Choi, QMCPy Developers

Aug 28 -- 29, 2025

Updated: Sep 8, 2025

**Requirements**:

* QMCPy: `pip install qmcpy==2.1`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

In [1]:
try:
    import qmcpy as qp
except ModuleNotFoundError:
    !pip install -q qmcpy

try:
    import parsl as pl
except ModuleNotFoundError:
    !pip install -q parsl

try:
    import testbook as tb
except ModuleNotFoundError:
    !pip install -q testbook

In [2]:
import sys
import os
import time
import inspect
import parsl as pl
from parsl.configs.htex_local import config

In [ ]:
start_time = time.time()
!(cd .. && make booktests)
sequential_time = time.time() - start_time


Generating missing booktest files...
cd test/booktests/ && python generate_test.py --check-missing
No missing test files found.

Notebook tests
set -e && \
	cd test/booktests/ && \
	rm -fr *.eps *.jpg *.pdf *.png *.part *.txt *.log prob_failure_gp_ci_plots && \
	PYTHONWARNINGS="ignore::UserWarning,ignore::DeprecationWarning,ignore::FutureWarning,ignore::ImportWarning" \
	python -W ignore -m coverage run --append --source=../../qmcpy/ -m unittest discover -s . -p "*.py" -v --failfast && \
	cd ../..
test_argonne_talk_2023_figures_notebook (tb_Argonne_2023_Talk_Figures.NotebookTests) ... skipped 'Runtime error'
test_mcqmc2022_article_figures_notebook (tb_MCQMC2022_Article_Figures.NotebookTests) ... skipped 'Runtime error'
test_mcqmc_2020_qmc_software_tutorial_notebook (tb_MCQMC_2020_QMC_Software_Tutorial.NotebookTests) ... skipped 'Runtime error'
test_purdue_talk_figures_notebook (tb_Purdue_Talk_Figures.NotebookTests) ... skipped 'Runtime error'
test_acm_toms_sorokin_2025_notebook (tb_ac

## 2. Parsl

1. Install and Configure Parsl
2. Run the tests in Parallel with Parsl

### 2.1 Install and Configure Parsl

In [ ]:
config.max_workers = 8  
if not pl.config:
    pl.load(config)
    print("Parsl loaded with memory limits")
else:
    print("Parsl already configured")

### 2.2 Create a Parsl Test Runner

In [ ]:
# Ensure the path to the booktests directory is included
sys.path.append(os.path.join(os.getcwd(), '..', 'test', 'booktests'))

In [ ]:
import parsl_test_runner
import inspect

# See only functions
print("Functions:")
functions = inspect.getmembers(parsl_test_runner, inspect.isfunction)
for name, func in functions:
    print(f"- {name}")
print("\n" + "="*50)

# Get help on specific function
print("Help for execute_parallel_tests:")
help(parsl_test_runner.execute_parallel_tests)

### 2.3 Run the Notebook in Parallel with Parsl

In [ ]:
start_time2 = time.time()
!(cd .. && make booktests-parallel)
parallel_time = time.time() - start_time2

In [ ]:
speedup = sequential_time / parallel_time
print(f"Sequential time:  {sequential_time:.2f} seconds")
print(f"Parallel time:    {parallel_time:.2f} seconds")
print(f"Speedup by Parsl: {speedup:.2f}")

In [ ]:
!uname -a
!nproc --all
!awk '/MemTotal/ {printf "%.2f GB\n", $2/1024/1024}' /proc/meminfo